# Core Basics 1: Train, Evaluate and Deploy a Classifier
In this lesson we will learn how to train, evaluate and deploy classifiers with Khiops.

We start by importing Khiops, some helper functions and saving the location of the Khiops `Samples` directory into a variable

In [ ]:
from os import path

from khiops import core as kh
from helper_functions import explorer_open, peek

samples_dir = kh.get_runner().samples_dir
print(f"Khiops samples directory located at {samples_dir}")

## Training a Classifier
We'll train a classifier for the `Iris` dataset. This is a classical dataset containing the data of different plants belonging to the genus _Iris_. It contains 150 records, 50 for each of three variants of _Iris_: _Setosa_, _Virginica_ and _Versicolor_. The records for each sample contain the length and width of its petal and sepal. The standard task for this dataset is to construct a classifier for the type of _Iris_ taking as inputs the length and width characteristics.

Now to train a classifier with Khiops we use two types of files:
- A plain-text delimited data file (for example a `csv` file)
- A _dictionary_ file which describes the schema of the above data table (`.kdic` file extension)


Let's save into variables the locations of these files for the `Iris` dataset and then take a look at their contents:

In [ ]:
iris_kdic = path.join(samples_dir, "Iris", "Iris.kdic")
iris_data_file = path.join(samples_dir, "Iris", "Iris.txt")

print("")
print(f"Iris dictionary file location: {iris_kdic}")
print("")
peek(iris_kdic)

print("")
print("")
print(f"Iris data location: {iris_data_file}")
print("")
peek(iris_data_file)

Note that the _Iris_ variant information is in the column `Class`. Now let's specify directory to save our results:

In [ ]:
iris_results_dir = path.join("exercises", "Iris")
print(f"Iris results directory: {iris_results_dir}")

We are now ready to train the classifier with the Khiops function `train_predictor`. This method returns a tuple containing the location of two files:
- the modeling report (`AllReports.khj`): A JSON file containing information such as the informativeness of each variable, those selected for the model and performance metrics.
- model's _dictionary_ file (`Modeling.kdic`): This file is an enriched version of the initial dictionary file that contains the model. It can be used to make predictions on new data.

In [ ]:
iris_report, iris_model_kdic = kh.train_predictor(
    iris_kdic,
    dictionary_name="Iris",
    data_table_path=iris_data_file,
    target_variable="Class",
    results_dir=iris_results_dir,
    max_trees=0,  # by default Khiops constructs 10 decision tree variables
)
print(f"Iris report file located at: {iris_report}")
print(f"Iris modeling dictionary file located at: {iris_model_kdic}")

You can verify that the result files were created in `iris_results_dir`. In the next sections, we'll use the file at `iris_report` to assess the models' performances and the file at `iris_model_kdic` to deploy it.

In [ ]:
# To take a look at the directory where the resulting files are stored
# explorer_open(iris_results_dir)

### Exercise

We'll repeat the examples on this notebook with the `Adult` dataset. It contains characteristics of the adult population in USA such as age, gender and education and its task is to predict the variable `class`, which indicates if the individual earns `more` or `less` than 50,000 dollars.

Let's start by putting into variables the paths for the `Adult` dataset:

In [ ]:
adult_kdic = path.join(samples_dir, "Adult", "Adult.kdic")
adult_data_file = path.join(samples_dir, "Adult", "Adult.txt")

#### Print the file locations and use the function `peek` to list their contents

In [ ]:
print("")
print(f"Adult dictionary file location: {adult_kdic}")
print("")
peek(adult_kdic)

print("")
print("")
print(f"Adult data location: {adult_data_file}")
print("")
peek(adult_data_file)

We now save the results directory for this exercise:

In [ ]:
adult_results_dir = path.join("exercises", "Adult")
print(f"Adult exercise results directory: {adult_results_dir}")

#### Train a classifier for the `Adult` database
Note the name of the target variable is `class` (**in lower case!**). Do not forget to set `max_trees=0`. Save the resulting file locations into the variables `adult_report` and `adult_model_kdic` and print them

In [ ]:
adult_report, adult_model_kdic = kh.train_predictor(
    adult_kdic,
    dictionary_name="Adult",
    data_table_path=adult_data_file,
    target_variable="class",
    results_dir=adult_results_dir,
    max_trees=0,
)
print(f"Adult report file located at: {adult_report}")
print(f"Adult modeling dictionary file located at: {adult_model_kdic}")

## Accessing a Classifiers' Basic Evaluation Metrics

We access the classifier's evaluation metrics by loading file at `iris_report` file with the Khiops function `read_analysis_results_file`:

In [ ]:
iris_results = kh.read_analysis_results_file(iris_report)
print(type(iris_results))

The resulting object is an instance of the `AnalysisResults` class. The model evaluation reports are stored in its `train_evaluation_report` and `test_evaluation_report` attributes which are of class `EvaluationReport`.

In [ ]:
iris_train_eval = iris_results.train_evaluation_report
iris_test_eval = iris_results.test_evaluation_report
print(type(iris_train_eval))
print(type(iris_test_eval))

We access the default predictor's metrics with the `get_snb_performance` method of the evaluation report objects:

In [ ]:
iris_train_performance = iris_train_eval.get_snb_performance()
iris_test_performance = iris_test_eval.get_snb_performance()

These objects are of class `PredictorPerformance`  and have `accuracy` and `auc` attributes for these metrics:

In [ ]:
print(f"Iris train accuracy: {iris_train_performance.accuracy}")
print(f"Iris test accuracy:  {iris_test_performance.accuracy}")
print("")
print(f"Iris train AUC: {iris_train_performance.auc}")
print(f"Iris test AUC:  {iris_test_performance.auc}")

### Exercise
#### Read the contents of the file at `adult_report` for the Adult analysis and print its type

In [ ]:
adult_results = kh.read_analysis_results_file(adult_report)
type(adult_results)

#### Save the evaluation reports of the `Adult` classification to the variables `adult_train_eval` and `adult_test_eval`

In [ ]:
adult_train_eval = adult_results.train_evaluation_report
adult_test_eval = adult_results.test_evaluation_report

#### Show the model's train and test accuracies and AUCs

In [ ]:
adult_train_performance = adult_train_eval.get_snb_performance()
adult_test_performance = adult_test_eval.get_snb_performance()
print(f"Adult train accuracy: {adult_train_performance.accuracy}")
print(f"Adult test accuracy:  {adult_test_performance.accuracy}")
print("")
print(f"Adult train AUC: {adult_train_performance.auc}")
print(f"Adult test AUC:  {adult_test_performance.auc}")

## Deploying a Classifier
We are going to deploy the `Iris` classifier we have just trained on the same dataset (normally we would do this on new data). We saved the model in the file `iris_model_kdic`. This file is usually large and incomprehensible, so you should know what you are doing before editing it. Just this time let's take a quick look at its contents:

In [ ]:
peek(iris_model_kdic, 25)

Note that the modeling dictionary contains 5 used variables:
- `Class` : The original target of the dataset
- `PredictedClass` : The class with the highest probability according to the model
- `ProbClassIris-setosa`, `ProbClassIris-versicolor`, `ProbClassIris-virginica`: The probabilities of each class according to the model

These will be the columns of the output table when deploying the model:

In [ ]:
iris_deployment_file = path.join(iris_results_dir, "iris_deployment.txt")
kh.deploy_model(
    iris_model_kdic,
    dictionary_name="SNB_Iris",
    data_table_path=iris_data_file,
    output_data_table_path=iris_deployment_file,
)

peek(iris_deployment_file)

### Exercise
#### Use the `deploy_model` function to deploy the model stored in the file at `adult_model_kdic`
Which columns are deployed?

In [ ]:
adult_deployment_file = path.join(adult_results_dir, "adult_deployment.txt")
kh.deploy_model(
    adult_model_kdic,
    dictionary_name="SNB_Adult",
    data_table_path=adult_data_file,
    output_data_table_path=adult_deployment_file,
)
peek(adult_deployment_file)